## Bibliotecas

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score

## Leitura do Dataset

In [12]:
dataset = pd.read_csv('/Users/jeanlima/Documents/GitHub/ElectricCurrentRecognition/CreatingDataset/ElectricCurrentDataset.txt')
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,Device
0,0.00,-0.18,-0.25,0.14,0.18,-0.14,0.07,-0.25,-0.04,-0.07,...,1.98,1.49,1.17,1.17,1.20,1.03,0.81,0.46,0.32,Notebook
1,0.07,-0.07,0.07,0.00,0.04,-0.07,0.14,-0.07,0.00,0.32,...,0.11,0.21,0.11,-0.04,-0.07,0.14,0.14,0.11,0.04,Notebook
2,0.07,0.04,-0.28,0.00,0.14,-0.25,-0.07,0.07,-0.04,0.11,...,0.00,0.14,-0.18,0.07,-0.28,0.11,-0.04,0.14,-0.11,Notebook
3,0.00,0.00,0.00,-0.18,0.60,1.42,2.23,1.84,1.66,1.24,...,-0.46,-0.25,-0.07,0.11,0.28,-0.07,0.00,0.32,-0.21,Notebook
4,1.59,1.42,1.20,1.35,1.31,1.10,0.60,0.18,0.00,-0.04,...,-0.07,-0.04,0.14,0.11,-0.07,0.14,0.18,-0.14,-0.11,Notebook


## Partição Holdout 70/30

In [13]:
y = dataset['Device']
dataset.drop(['Device'], axis=1, inplace=True)

X_train, X_test, Y_train, Y_test = train_test_split(dataset, y, test_size=0.30)

## Treinamento knn; k=5

In [5]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

## Teste do modelo

In [14]:
previsaokNN = knn.predict(X_test)

## Matriz de Confusão

In [17]:
print(confusion_matrix(Y_test, previsaokNN))

[[62  0]
 [ 0 59]]


## Métricas de Desempenho

### Acurácia

In [8]:
accuracy_score(Y_test, previsaokNN)

1.0

### Precisão

In [9]:
precision_score(Y_test, previsaokNN, pos_label='Notebook')

1.0

### F1-Score

In [10]:
f1_score(Y_test, previsaokNN, pos_label='Notebook')

1.0